### Create EEZ map mask

In [285]:
# - Read in tropical Pacific EEZ shape file
eezs = gpd.read_file(
    dpath + 'EEZs/World_EEZ_v9_20161021_HR_0_360/World_EEZ_v9_2016_HR_0_360.shp')
eezs = eezs[['PolygonID', 'GeoName', 'Sovereign1', 'geometry']]
nuctries = ['Palau','Micronesia','Papua New Guinea','Solomon Islands','Nauru','Marshall Islands','Tuvalu','Kiribati'];
nueezs = eezs[eezs['Sovereign1'].isin(nuctries)]

In [287]:
# - Get EEZ outlines only (no islands in middle of EEZs)
eezs['borders'] = ''
for irow in range(0,len(eezs)):
    try: 
        eezs['borders'].iloc[irow] = Polygon(eezs.iloc[irow]['geometry'].exterior.coords)
    except: 
        eezs['borders'].iloc[irow] = eezs.iloc[irow]['geometry']
eezs = eezs.set_geometry('borders')
nueezs['borders'] = ''
for irow in range(0,len(nueezs)):
    nueezs['borders'].iloc[irow] = Polygon(nueezs.iloc[irow]['geometry'].exterior.coords)
nueezs = nueezs.set_geometry('borders')

In [24]:
# - Create EEZ mask over current lon-lat grid (this will naturally exclude ETP countries)
nueeznewnumbers = list(range(0,len(nueezs)))
nueeznames = nueezs.Sovereign1.values
# Detailed Kiribati names were (look at zero-indexed row # of names in eez df): 
nueeznames[5] = 'Kiribati (Phoenix Islands)'
nueeznames[9] = 'Kiribati (Line Islands)'
nueeznames[10] = 'Kiribati (Gilbert Islands)'
nueezabbrevs = [w[0:4] for w in nueeznames]
# nueezregionmask contains all nu EEZs in the loaded shapefile
nueezregionmask = regionmask.Regions_cls('mask', nueeznewnumbers, nueeznames, nueezabbrevs, nueezs.geometry.reset_index(drop=True))
# nueezmask contains only EEZs within the current lon-lat grid defined by lon,lat arguments below
nueezmask = nueezregionmask.mask(lon,lat,wrap_lon=False)
# - Create a table that maps between eez number in eezmask and eeznames
df_nuenn = pd.DataFrame({'names':nueeznames, 'numbers':nueeznewnumbers}) #df_enn = df_eeznamesnumbers